In [42]:
import pymysql.cursors

db = pymysql.connect(host='13.230.161.175',
                             user='public-root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

def search_db(keyword):
    try:
        cursor = db.cursor()#建立資料庫游標
        
        sql = ("select distinct(game_name) \
                from game left JOIN (game_has_type inner JOIN type using (type_ID)) using (game_ID)\
                where type_name like '%"+keyword+"%' ")#下指令
        
        cursor.execute(sql)
        rows = cursor.fetchall()
        return str([i['game_name'] for i in rows])
    except Exception as e:
        print("Exeception occured:{}".format(e))
    finally:
        db.close()

In [37]:
import json
from flask import Flask, request, make_response, jsonify
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pymysql.cursors

app = Flask(__name__)


@app.route("/")
def verify():
    return "Hello world", 200


@app.route('/', methods=['POST'])#Dialogflow端
def webhook():
    req = request.get_json(silent=True, force=True)
    print(req)
    if req['queryResult']['parameters']['game_type'] != None:
        keyword = req['queryResult']['parameters']['game_type']
    res = {"fulfillmentText": search_db(str(keyword))}
    res = json.dumps(res,indent = 4)
    print(res)
    return res


if __name__ == '__main__':
    db = pymysql.connect(host='13.230.161.175',#連結MySQL
                             user='public-root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    app.run(port=5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Dec/2018 16:28:32] "POST / HTTP/1.1" 200 -


{'responseId': 'c255e16a-d771-4121-804b-b6674f3148f2', 'queryResult': {'queryText': '恐怖', 'action': 'action', 'parameters': {'game_type': '驚悚'}, 'allRequiredParamsPresent': True, 'fulfillmentMessages': [{'text': {'text': ['']}}], 'intent': {'name': 'projects/deepbowl-e7505/agent/intents/71659b1b-175b-465a-b508-067b18a7e475', 'displayName': 'GameChatBot'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/deepbowl-e7505/agent/sessions/b7045468-a131-3321-8af9-585e4a41a25f'}
['STEINS;GATE', 'Mirror', 'Russian Fishing 4', '東方憑依華\u3000～ Antinomy of Common Flowers.', 'World of Tanks Blitz', 'Danganronpa V3: Killing Harmony', 'SENRAN KAGURA ESTIVAL VERSUS', 'Coming Out on Top', 'We Were Here Too', 'INSIDE']
{
    "fulfillmentText": "['STEINS;GATE', 'Mirror', 'Russian Fishing 4', '\u6771\u65b9\u6191\u4f9d\u83ef\\u3000\uff5e Antinomy of Common Flowers.', 'World of Tanks Blitz', 'Danganronpa V3: Killing Harmony', 'SEN